In [3]:
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import timedelta, datetime


In [41]:
class StockData:
    def __init__(self, ticker):
        self.ticker = ticker
        self.df = self.get_data(ticker)
        self.volatility_range = self.get_volatility_range()
        self.filtered_df = self.filter_volatility_range()
        self.bounds = self.get_bounds_for_running_prob()
        self.curr_val = self.df.iloc[-1]['Close']
        self.calls, self.puts, self.expiration_date = self.get_options_for_next_week()
        
    def per_to_val(self, per):
        return (per * 0.01 + 1) * self.curr_val
    
    def val_to_per(self, val):
        return (val / self.curr_val - 1) * 100
    
    def get_bounds_for_percentile(self, percentile):
        lower_bound, upper_bound = self.bounds[self.bounds['Percentile'] == percentile]['Bounds'][0]
        return lower_bound, upper_bound
    
    def get_options(self, lower_bound, upper_bound):
        closest_upper = self.calls.iloc[(self.calls['strike'] - upper_bound).abs().idxmin()]
        closest_lower = self.puts.iloc[(self.puts['strike'] - lower_bound).abs().idxmin()]
        return closest_upper['strike'], closest_upper['lastPrice'], closest_lower['strike'], closest_lower['lastPrice']
    
    def get_empirical_percent(self, lower_bound, upper_bound):
        return self.get_percent_for_bound(lower_bound, upper_bound)
    
    def get_options_for_next_week(self):
        tk = yf.Ticker(self.ticker)
        next_week_date = datetime.now() + timedelta(days=7)
        nearest_expiration = min(tk.options, key=lambda date: abs(datetime.strptime(date, "%Y-%m-%d") - next_week_date))
        print(tk.options)
        options_data = tk.option_chain(nearest_expiration)
        print(options_data)
        calls_data = options_data.calls
        puts_data = options_data.puts
        return calls_data, puts_data, nearest_expiration
    
    def get_data(self, ticker, start_date= '2000-01-01', end_date = pd.to_datetime('today').strftime('%Y-%m-%d'), interval='1wk', period=None):
        if period is not None:
            df = yf.download(ticker, period=period, interval=interval)
        else:
            df = yf.download(ticker, start=start_date, end=end_date, interval=interval)
        window_size = 3
        log_returns = np.log(df['Adj Close'] / df['Adj Close'].shift(1))
        rolling_std_dev = log_returns.rolling(window=window_size).std()
        rolling_annualized_volatility = rolling_std_dev * np.sqrt(52)
        df['Volatility'] = rolling_annualized_volatility
        df['weekly_change'] = df['Adj Close'].pct_change() * 100
        return df
    
    def get_volatility_range(self, num_std_devs=1):
        curr_volatility = self.df['Volatility'][-1]
        std = self.df['Volatility'].std()
        return (curr_volatility - num_std_devs * std, curr_volatility + num_std_devs * std)
    
    def filter_volatility_range(self):
        return self.df[(self.df['Volatility'] >= self.volatility_range[0]) & (self.df['Volatility'] <= self.volatility_range[1])]
    
    def get_bounds_for_running_prob(self):
        bounds = {}
        for percentile in range(5, 50, 5):
            percentile = percentile / 100
            range_lower, range_upper = self.filtered_df['weekly_change'].quantile(percentile), self.filtered_df['weekly_change'].quantile(1-percentile)
            bounds[int((1-(percentile * 2)) * 100)] = (round(range_lower,2), round(range_upper,2))
        bounds_df = pd.DataFrame(bounds.items(), columns=['Percentile', 'Bounds'])
        return bounds_df
    
    def get_percent_for_bound(self, lower_bound, upper_bound):
        return len(self.filtered_df[(self.filtered_df['weekly_change'] >= lower_bound) & (self.filtered_df['weekly_change'] <= upper_bound)]) / len(self.filtered_df) * 100
    
    def calculate_option_selling_profit_loss(self, put_strike, call_strike, current_price, put_premium, call_premium):
        # Calculate profit/loss for the sold call option
        if current_price > call_strike:
            call_profit_loss = call_premium - (current_price - call_strike)
        else:
            call_profit_loss = call_premium  # Retain premium if not exercised
    
        # Calculate profit/loss for the sold put option
        if current_price < put_strike:
            put_profit_loss = put_premium - (put_strike - current_price)
        else:
            put_profit_loss = put_premium  # Retain premium if not exercised
    
        total_profit_loss = call_profit_loss + put_profit_loss
        return call_profit_loss, put_profit_loss, total_profit_loss
    
    
    def pick_option_given_confidence_percent(self, lower_bound, upper_bound):
        lower_bound_val, upper_bound_val = self.per_to_val(upper_bound), self.per_to_val(lower_bound)
        return self.get_options(lower_bound_val, upper_bound_val)

In [43]:
aapl = StockData('AAPL')


[*********************100%%**********************]  1 of 1 completed
C:\Users\noama\AppData\Local\Temp\ipykernel_23300\1723322222.py:54: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  curr_volatility = self.df['Volatility'][-1]


('2024-04-19', '2024-04-26', '2024-05-03', '2024-05-10', '2024-05-17', '2024-05-24', '2024-05-31', '2024-06-21', '2024-07-19', '2024-08-16', '2024-09-20', '2024-10-18', '2024-11-15', '2024-12-20', '2025-01-17', '2025-03-21', '2025-06-20', '2025-09-19', '2025-12-19', '2026-01-16', '2026-06-18', '2026-12-18')
Options(calls=         contractSymbol             lastTradeDate  strike  lastPrice  bid  \
0   AAPL240426C00100000 2024-04-10 19:58:13+00:00   100.0      68.20  0.0   
1   AAPL240426C00110000 2024-04-11 19:19:05+00:00   110.0      64.40  0.0   
2   AAPL240426C00125000 2024-04-12 16:32:39+00:00   125.0      50.72  0.0   
3   AAPL240426C00130000 2024-04-11 19:18:19+00:00   130.0      44.86  0.0   
4   AAPL240426C00135000 2024-04-08 15:09:17+00:00   135.0      34.57  0.0   
5   AAPL240426C00140000 2024-04-12 14:32:28+00:00   140.0      36.70  0.0   
6   AAPL240426C00145000 2024-04-11 19:22:11+00:00   145.0      30.00  0.0   
7   AAPL240426C00149000 2024-04-12 15:24:05+00:00   149.0    

In [49]:
#TODO calc strategy where: given a stock, calc it's 90% bound then calc the options to buy given the bound.
# Then a function that given the options i bought last week calc how much i earned/lost 
#then do for all nasdaq and check for all confidence percent

sum_profit = 0
data = []
for percentile, bounds in aapl.bounds:
    lower_bound, upper_bound = bounds
    put_strike, put_premium, call_strike, call_premium = aapl.pick_option_given_confidence_percent(lower_bound, upper_bound)
    print(f'put_strike: {put_strike}, put_premium: {put_premium}, call_strike: {call_strike}, call_premium: {call_premium}')
    data.append({
        'ticker': aapl.ticker,
        'Percentile': percentile,
        'Lower Bound': lower_bound,
        'Upper Bound': upper_bound,
        'Put Strike': put_strike,
        'Put Premium': put_premium,
        'Call Strike': call_strike,
        'Call Premium': call_premium
    })
    call_profit_loss, put_profit_loss, total_profit_loss = aapl.calculate_option_selling_profit_loss(put_strike, call_strike, 200, put_premium, call_premium)
    print(f'profit {total_profit_loss} for bound: {put_strike} - {call_strike}')

print(sum_profit)



ValueError: too many values to unpack (expected 2)

In [44]:
aapl.bounds

,Percentile,Bounds
0,90,"(-5.66, 6.98)"
1,80,"(-4.26, 5.38)"
2,70,"(-3.37, 4.37)"
3,60,"(-2.51, 3.59)"
4,50,"(-1.85, 2.97)"
5,40,"(-1.22, 2.32)"
6,30,"(-0.69, 1.84)"
7,19,"(-0.24, 1.42)"
8,9,"(0.18, 0.99)"
